In [1]:
import json
import os
from pathlib import Path
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

In [2]:
project_name = "sports_cv_project_classification"

In [3]:
root_dir = Path().cwd().parent.parent
os.chdir(root_dir)

In [4]:
logs_dict = {}
logs_path = os.path.join(root_dir, "logs", project_name)
for experiment in os.listdir(logs_path):
    experiment_metrics = {}
    for part in ["train", "test", "val"]:
        metrics_path = os.path.join(logs_path, experiment, part, "metrics.json")

        if os.path.exists(metrics_path):
            with open(metrics_path, "r") as f:
                metrics = json.load(f)
            experiment_metrics[part] = metrics
        else:
            print(f"Skipping, not found: {metrics_path}")
    logs_dict[experiment] = experiment_metrics


Skipping, not found: /workspace/Vision-models-pytorch/SportsClassification/logs/sports_cv_project_classification/alexnet-base/test/metrics.json
Skipping, not found: /workspace/Vision-models-pytorch/SportsClassification/logs/sports_cv_project_classification/alexnet-normalized-base/test/metrics.json
Skipping, not found: /workspace/Vision-models-pytorch/SportsClassification/logs/sports_cv_project_classification/alexnet-augmented-base/test/metrics.json


In [5]:
logs_dict["alexnet-base"]["train"].keys()

dict_keys(['loss', 'accuracy', 'top-5-accuracy', 'epoch_loss', 'epoch_accuracy', 'epoch_top-5-accuracy'])

## Training metrics per steps

In [6]:
# Flatten dict into dataframe
rows = []
for exp, parts in logs_dict.items():
    for metric, values in parts["train"].items():
        for epoch, val in enumerate(values, 1):
            rows.append({
                "experiment": exp,
                "metric": metric,
                "epoch": epoch,
                "value": val
            })

df = pd.DataFrame(rows)

experiments = df["experiment"].unique()
metrics = ['loss', 'accuracy', 'top-5-accuracy']

# Assign consistent colors using Plotly Express palette
colors = px.colors.qualitative.Plotly
color_map = {exp: colors[i % len(colors)] for i, exp in enumerate(experiments)}

fig = go.Figure()

# Add all traces
for metric in metrics:
    for exp in experiments:
        subset = df[(df["metric"] == metric) & (df["experiment"] == exp)]
        fig.add_trace(go.Scatter(
            x=subset["epoch"],
            y=subset["value"],
            mode="lines",
            name=exp,
            line=dict(color=color_map[exp]),
            visible=(metric == "loss")  # only show loss at first
        ))

# Visibility mapping for dropdown
n_experiments = len(experiments)
visibility_map = []
for i, metric in enumerate(metrics):
    visible = [False] * (len(metrics) * n_experiments)
    start = i * n_experiments
    end = start + n_experiments
    for j in range(start, end):
        visible[j] = True
    visibility_map.append(visible)

# Dropdown menu
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": metric,
                    "method": "update",
                    "args": [
                        {"visible": vis},
                        {"title": f"Training {metric.capitalize()}"}
                    ],
                }
                for metric, vis in zip(metrics, visibility_map)
            ],
            "direction": "down",
            "showactive": True,
        }
    ],
    title="Training Loss",
    xaxis_title="Steps",
    yaxis_title="Value"
)

fig.show()


## Metrics per epoch for train and validation datasets

In [7]:
# Flatten dict into dataframe
rows = []
datasets = ["train", "val"]
for exp, parts in logs_dict.items():
    for metric, values in parts["train"].items():
        for epoch, val in enumerate(values, 1):
            rows.append({
                "dataset": "train",
                "experiment": exp,
                "metric": metric,
                "epoch": epoch,
                "value": val
            })
    for metric, values in parts["val"].items():
        for epoch, val in enumerate(values, 1):
            rows.append({
                "dataset": "val",
                "experiment": exp,
                "metric": metric,
                "epoch": epoch,
                "value": val
            })

df = pd.DataFrame(rows)

experiments = df["experiment"].unique()
metrics = ['epoch_loss', 'epoch_accuracy', 'epoch_top-5-accuracy']

# Assign consistent colors
colors = px.colors.qualitative.Plotly
color_map = {exp: colors[i % len(colors)] for i, exp in enumerate(experiments)}

fig = go.Figure()

# Add traces for all metrics and datasets
for metric in metrics:
    for exp in experiments:
        for dataset in datasets:
            subset = df[(df["metric"] == metric) & 
                        (df["experiment"] == exp) & 
                        (df["dataset"] == dataset)]
            fig.add_trace(go.Scatter(
                x=subset["epoch"],
                y=subset["value"],
                mode="lines",
                name=f"{exp} ({dataset})",
                line=dict(
                    color=color_map[exp],
                    dash='dot' if dataset == "val" else 'solid'
                ),
                visible=(metric == 'epoch_loss')  # initially show loss
            ))

# Build visibility mapping for dropdown
n_experiments = len(experiments)
visibility_map = []

for metric in metrics:
    visible = [False] * (len(metrics) * n_experiments * len(datasets))
    start_idx = metrics.index(metric) * n_experiments * len(datasets)
    end_idx = start_idx + n_experiments * len(datasets)
    for i in range(start_idx, end_idx):
        visible[i] = True
    visibility_map.append(visible)

# Create dropdown menu
fig.update_layout(
    updatemenus=[{
        "buttons": [
            {
                "label": metric,
                "method": "update",
                "args": [
                    {"visible": vis},
                    {"title": f"{metric.replace('epoch_', '').capitalize()} per Experiment"}
                ]
            } for metric, vis in zip(metrics, visibility_map)
        ],
        "direction": "down",
        "showactive": True,
    }],
    title="Training Loss",
    xaxis_title="Epoch",
    yaxis_title="Value"
)

fig.show()
